In [6]:
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm, trange

In [3]:
train_path = './bgc_train.json'
dev_path = './bgc_val.json'
test_path = './bgc_test.json'

train_data, val_data, test_data = [], [], []
with open(train_path) as fd:
    for line in fd.readlines():
        train_data.append(json.loads(line))
with open(dev_path) as fd:
    for line in fd.readlines():
        val_data.append(json.loads(line))
with open(test_path) as fd:
    for line in fd.readlines():
        test_data.append(json.loads(line))
        

In [4]:
train_data[0]

{'token': 'Title: The New York Times Daily Crossword Puzzles: Thursday, Volume 1. Text: Monday’s Crosswords Do with EaseTuesday’s Crosswords Not a BreezeWednesday’s Crosswords Harder StillThursday’s Crosswords Take Real SkillFriday’s Crosswords — You’ve Come This Far…Saturday’s Crosswords — You’re a Star!For millions of people, the New York Times crossword puzzles are as essential to each day as the first cup of coffee in the morning. Now, for the first time ever, these premier puzzles are available in six clever installments. With each day of the week, the puzzles increase gradually in skill level; Monday’s the easiest, but Saturday’s sure to challenge! Push your mental muscles a little harder each day with America’s favorite sophisticated — and fun — pastime: the New York Times crossword puzzles!The legendary Eugene T. Maleska was crossword editor of The New York Times from 1977 to 1993.',
 'label': ['Nonfiction', 'Games']}

In [5]:
labels = []
for data in train_data:
    labels.append(len(data['label']))
for data in val_data:
    labels.append(len(data['label']))
for data in test_data:
    labels.append(len(data['label']))

print('max label length: ', max(labels))
print('min label length: ', min(labels))
print('mean label length: ', np.mean(labels))

max label length:  11
min label length:  1
mean label length:  3.009391255141794


In [7]:
import nltk
import re

def filter_non_tokens(sentence):
    # Tokenize the sentence
    tokens = nltk.word_tokenize(sentence)
    
    # Define a regular expression pattern to match punctuation marks
    punctuation_pattern = re.compile(r'^\W*$')
    
    # Filter out punctuation marks
    filtered_tokens = [token for token in tokens if not punctuation_pattern.match(token)]
    
    return filtered_tokens

def filter_stop_words(tokens):    
    # Filter out stop words
    filtered_tokens = [token for token in tokens if token not in nltk.corpus.stopwords.words('english')]
    
    return filtered_tokens

for i in trange(len(train_data)):
    # train_data[i]['token'] = [str.lower(t).replace(',', '').replace('.', '').replace(":", "") for t in train_data[i]['token'].split()]
    train_data[i]['token'] = filter_non_tokens(train_data[i]['token'])
    train_data[i]['token'] = [str.lower(t).replace(',', '').replace('.', '').replace(":", "") for t in train_data[i]['token']]
    train_data[i]['token'] = filter_stop_words(train_data[i]['token'])

    # train_data[i]['label'] = [label_mapping[l] for l in train_data[i]['label']]

for i in trange(len(val_data)):
    # val_data[i]['token'] = [str.lower(t).replace(',', '').replace('.', '').replace(":", "") for t in val_data[i]['token'].split()]
    val_data[i]['token'] = filter_non_tokens(val_data[i]['token'])
    val_data[i]['token'] = [str.lower(t).replace(',', '').replace('.', '').replace(":", "") for t in val_data[i]['token']]

    val_data[i]['token'] = filter_stop_words(val_data[i]['token'])
    # val_data[i]['token'] = [str.lower(t).replace(',', '').replace('.', '') for t in val_data[i]['token']]

for i in trange(len(test_data)):
    # test_data[i]['token'] = [str.lower(t).replace(',', '').replace('.', '').replace(":", "") for t in test_data[i]['token'].split()]
    test_data[i]['token'] = filter_non_tokens(test_data[i]['token'])
    test_data[i]['token'] = [str.lower(t).replace(',', '').replace('.', '').replace(":", "") for t in test_data[i]['token']]
    test_data[i]['token'] = filter_stop_words(test_data[i]['token'])
    # test_data[i]['token'] = [str.lower(t).replace(',', '').replace('.', '') for t in test_data[i]['token']]
    # test_data[i]['label'] = [label_mapping[l] for l in test_data[i]['label']]

100%|██████████| 18394/18394 [01:40<00:00, 182.31it/s]


In [9]:
train_data[0]

{'token': ['title',
  'new',
  'york',
  'times',
  'daily',
  'crossword',
  'puzzles',
  'thursday',
  'volume',
  '1',
  'text',
  'monday',
  'crosswords',
  'easetuesday',
  'crosswords',
  'breezewednesday',
  'crosswords',
  'harder',
  'stillthursday',
  'crosswords',
  'take',
  'real',
  'skillfriday',
  'crosswords',
  'come',
  'far…saturday',
  'crosswords',
  'star',
  'millions',
  'people',
  'new',
  'york',
  'times',
  'crossword',
  'puzzles',
  'essential',
  'day',
  'first',
  'cup',
  'coffee',
  'morning',
  'first',
  'time',
  'ever',
  'premier',
  'puzzles',
  'available',
  'six',
  'clever',
  'installments',
  'day',
  'week',
  'puzzles',
  'increase',
  'gradually',
  'skill',
  'level',
  'monday',
  'easiest',
  'saturday',
  'sure',
  'challenge',
  'push',
  'mental',
  'muscles',
  'little',
  'harder',
  'day',
  'america',
  'favorite',
  'sophisticated',
  'fun',
  'pastime',
  'new',
  'york',
  'times',
  'crossword',
  'puzzles',
  'legendar

In [10]:
# dump them into json files
with open('./bgc_train.json', 'w') as fd:
    for data in train_data:
        json.dump(data, fd)
        fd.write('\n')

with open('./bgc_val.json', 'w') as fd:
    for data in val_data:
        json.dump(data, fd)
        fd.write('\n')

with open('./bgc_test.json', 'w') as fd:
    for data in test_data:
        json.dump(data, fd)
        fd.write('\n')

In [17]:
with open('./bgc.taxonomy', 'r') as fd:
    taxonomy = fd.readlines()

for i in range(len(taxonomy)):
    taxonomy[i] = '\t'.join([label_mapping[l] if l != 'Root' else l for l in taxonomy[i].strip('\n').split('\t')])

# dump the taxonomy into a file
with open('./bgc.taxonomy', 'w') as fd:
    for line in taxonomy:
        fd.write(line + '\n')

In [11]:
from collections import Counter
labels = []
vocabs = []

for data in train_data:
    labels.extend(data['label'])
    vocabs.extend(data['token'])

# count the frequency of each label and vocab
label_count = Counter(labels)
vocab_count = Counter(vocabs)

# sort the label and vocab by their frequency
label_count = sorted(label_count.items(), key=lambda x: x[1], reverse=True)
vocab_count = sorted(vocab_count.items(), key=lambda x: x[1], reverse=True)
# save as vocab_bgc/label.dict and vocab_bgc/vocab.dict

if not os.path.exists('./vocab_bgc'):
    os.mkdir('./vocab_bgc')

with open('./vocab_bgc/label.dict', 'w') as fd:
    for label, count in label_count:
        fd.write(label + '\t' + str(count) + '\n')

with open('./vocab_bgc/vocab.dict', 'w') as fd:
    for vocab, count in vocab_count:
        fd.write(vocab + '\t' + str(count) + '\n')